Backend Task: Strategy Backtester Engine : Executed Code Below

In [2]:

pip install requests pandas ts

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\SHREYA\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
#fetching data from Binance API
import requests
import pandas as pd
import datetime

def fetch_binance_data(symbol="BTCUSDT", interval="1m", limit=5000):
    url = f"https://api.binance.com/api/v3/klines"
    params={"symbol": symbol, "interval":interval, "limit":limit}
    response = requests.get(url, params=params)
    data = response.json()

    df = pd.DataFrame(data, columns=[
        "timestap", "open", "high", "low", "close", "volume"
    ])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("timestamp", inplace=True)
    df = df[["open", "high", "low", "close", "volume"]].astype(float)
    return df


Strategy 1: MACD EMA & Strategies

In [4]:
pip install ta

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\SHREYA\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import ta.trend as trend

def apply_macd_strategy(df):
    macd = trend.MACD(df['close'])
    df['macd_line'] = macd.macd()
    df['macd_ema'] = df['macd_line'].ewm(span=9).mean()

    df['signal'] = 0
    df.loc[(df['macd_line'] > df['macd_ema']) & (df['macd_line'].shift() <= df['macd_ema'].shift()), 'signal'] = 1
    df.loc[(df['macd_line'] < df['macd_ema']) & (df['macd_line'].shift() >= df['macd_ema'].shift()), 'signal'] = -1

    return df


Strategy 2 : RSI + EMA(21) Strategy

In [6]:
def apply_rsi_ema_strategy(df):
    df['rsi'] = RSIIndicator(df['close'], window=14).rsi()
    df['ema_21'] = df['close'].ewm(span=21).mean()

    df['signal'] = 0
    df.loc[(df['rsi'] > 30) & (df['rsi'].shift() <= 30) & (df['close'] > df['ema_21']), 'signal'] = 1
    df.loc[(df['rsi'] < 70) & (df['rsi'].shift() >= 70) | (df['close'] < df['ema_21']), 'signal'] = -1

    return df

In [7]:
def backtest_strategy(df, strategy_name):
    trades = []
    in_position = False
    entry_price = 0
    entry_time = None

    for i in range(len(df)):
        signal = df['signal'].iloc[i]
        time = df.index[i]
        price = df['close'].iloc[i]

        if signal == 1 and not in_position:
            in_position = True
            entry_price = price
            entry_time = time

        elif signal == -1 and in_position:
            exit_price = price
            exit_time = time
            pnl = exit_price - entry_price
            status = "Win" if pnl > 0 else "Loss"
            trades.append([
                entry_time, entry_price,
                exit_time, exit_price,
                strategy_name, pnl, status
            ])
            in_position = False

    for t in trades:
        print("Row Length:", len(t), "|", t)

    return pd.DataFrame(trades, columns=[
        "Entry Time", "Entry Price", "Exit Time",
        "Exit Price", "Strategy", "PnL", "Status"
    ])


In [8]:
if __name__ == "__main__":
    df = fetch_binance_data()

    # MACD Strategy
    macd_df = apply_macd_strategy(df.copy())
    macd_trades = backtest_strategy(macd_df, "MACD-EMA")

    # RSI Strategy
    rsi_df = apply_rsi_ema_strategy(df.copy())
    rsi_trades = backtest_strategy(rsi_df, "RSI-EMA")

    # Combine and Save
    all_trades = pd.concat([macd_trades, rsi_trades])
    print(all_trades)
    all_trades.to_csv("strategy_backtest_results.csv", index=False)


ValueError: 6 columns passed, passed data had 12 columns

Actual Executed Code Below

In [ ]:
import requests
import pandas as pd
from ta.trend import MACD


            Entry Time  Entry Price           Exit Time  Exit Price  Strategy  \
0  2025-06-02 21:39:00    104791.93 2025-06-02 22:10:00   105071.99  MACD-EMA   
1  2025-06-02 22:14:00    105228.43 2025-06-02 22:33:00   105677.10  MACD-EMA   
2  2025-06-02 22:52:00    105684.74 2025-06-02 22:56:00   105659.68  MACD-EMA   
3  2025-06-02 23:02:00    105772.41 2025-06-02 23:05:00   105666.48  MACD-EMA   
4  2025-06-02 23:20:00    105847.39 2025-06-02 23:27:00   105786.42  MACD-EMA   
5  2025-06-02 23:44:00    105905.66 2025-06-02 23:53:00   105879.35  MACD-EMA   
6  2025-06-03 00:09:00    105875.05 2025-06-03 00:27:00   106122.02  MACD-EMA   
7  2025-06-03 00:46:00    106316.65 2025-06-03 00:52:00   106248.17  MACD-EMA   
8  2025-06-03 01:02:00    106311.44 2025-06-03 01:11:00   106202.68  MACD-EMA   
9  2025-06-03 01:21:00    106275.93 2025-06-03 01:32:00   106242.30  MACD-EMA   
10 2025-06-03 01:47:00    106289.37 2025-06-03 02:06:00   106313.91  MACD-EMA   
11 2025-06-03 02:31:00    10

In [ ]:
from ta.momentum import RSIIndicator

# 1. Fetch Data from Binance
def fetch_binance_data(symbol="BTCUSDT", interval="1m", limit=5000):
    url = "https://api.binance.com/api/v3/klines"
    params = {"symbol": symbol, "interval": interval, "limit": limit}
    response = requests.get(url, params=params)
    data = response.json()

    df = pd.DataFrame(data, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "_", "_", "_", "_", "_", "_"
    ])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("timestamp", inplace=True)
    df = df[["open", "high", "low", "close", "volume"]].astype(float)
    return df


In [9]:

# 2. MACD Strategy
def apply_macd_strategy(df):
    macd = MACD(df['close'])
    df['macd_line'] = macd.macd()
    df['macd_ema'] = df['macd_line'].ewm(span=9).mean()

    df['signal'] = 0
    df.loc[
        (df['macd_line'] > df['macd_ema']) &
        (df['macd_line'].shift() <= df['macd_ema'].shift()), 'signal'
    ] = 1
    df.loc[
        (df['macd_line'] < df['macd_ema']) &
        (df['macd_line'].shift() >= df['macd_ema'].shift()), 'signal'
    ] = -1
    return df


In [10]:

# 3. RSI + EMA Strategy
def apply_rsi_ema_strategy(df):
    df['rsi'] = RSIIndicator(df['close'], window=14).rsi()
    df['ema_21'] = df['close'].ewm(span=21).mean()

    df['signal'] = 0
    df.loc[
        (df['rsi'] > 30) & (df['rsi'].shift() <= 30) &
        (df['close'] > df['ema_21']), 'signal'
    ] = 1
    df.loc[
        ((df['rsi'] < 70) & (df['rsi'].shift() >= 70)) |
        (df['close'] < df['ema_21']), 'signal'
    ] = -1
    return df


In [11]:

# 4. Backtesting Function
def backtest_strategy(df, strategy_name):
    trades = []
    in_position = False
    entry_price = 0
    entry_time = None

    for i in range(len(df)):
        signal = df['signal'].iloc[i]
        time = df.index[i]
        price = df['close'].iloc[i]

        if signal == 1 and not in_position:
            in_position = True
            entry_price = price
            entry_time = time

        elif signal == -1 and in_position:
            exit_price = price
            exit_time = time
            pnl = exit_price - entry_price
            status = "Win" if pnl > 0 else "Loss"
            trades.append([
                entry_time, entry_price,
                exit_time, exit_price,
                strategy_name, pnl, status
            ])
            in_position = False

    return pd.DataFrame(trades, columns=[
        "Entry Time", "Entry Price", "Exit Time",
        "Exit Price", "Strategy", "PnL", "Status"
    ])


In [13]:

# 5. Pipeline
if __name__ == "__main__":
    df = fetch_binance_data()

    macd_df = apply_macd_strategy(df.copy())
    macd_trades = backtest_strategy(macd_df, "MACD-EMA")


    rsi_df = apply_rsi_ema_strategy(df.copy())
    rsi_trades = backtest_strategy(rsi_df, "RSI-EMA")


    all_trades = pd.concat([macd_trades, rsi_trades])
    print(all_trades)
    all_trades.to_csv("strategy_backtest_results.csv", index=False)


ValueError: 6 columns passed, passed data had 12 columns

In [ ]:
import requests
import pandas as pd
from ta.trend import MACD
from ta.momentum import RSIIndicator


            Entry Time  Entry Price           Exit Time  Exit Price  Strategy  \
0  2025-06-02 21:28:00    104792.62 2025-06-02 21:32:00   104721.85  MACD-EMA   
1  2025-06-02 21:38:00    104727.15 2025-06-02 22:11:00   105129.88  MACD-EMA   
2  2025-06-02 22:14:00    105228.43 2025-06-02 22:33:00   105677.10  MACD-EMA   
3  2025-06-02 22:52:00    105684.74 2025-06-02 22:56:00   105659.68  MACD-EMA   
4  2025-06-02 23:02:00    105772.41 2025-06-02 23:05:00   105666.48  MACD-EMA   
5  2025-06-02 23:20:00    105847.39 2025-06-02 23:27:00   105786.42  MACD-EMA   
6  2025-06-02 23:44:00    105905.66 2025-06-02 23:53:00   105879.35  MACD-EMA   
7  2025-06-03 00:09:00    105875.05 2025-06-03 00:27:00   106122.02  MACD-EMA   
8  2025-06-03 00:46:00    106316.65 2025-06-03 00:52:00   106248.17  MACD-EMA   
9  2025-06-03 01:02:00    106311.44 2025-06-03 01:11:00   106202.68  MACD-EMA   
10 2025-06-03 01:21:00    106275.93 2025-06-03 01:32:00   106242.30  MACD-EMA   
11 2025-06-03 01:47:00    10

In [ ]:

# 1. Fetch Data from Binance
def fetch_binance_data(symbol="BTCUSDT", interval="1m", limit=5000):
    url = "https://api.binance.com/api/v3/klines"
    params = {"symbol": symbol, "interval": interval, "limit": limit}
    response = requests.get(url, params=params)
    data = response.json()

    df = pd.DataFrame(data, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "_", "_", "_", "_", "_", "_"
    ])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("timestamp", inplace=True)
    df = df[["open", "high", "low", "close", "volume"]].astype(float)
    return df


In [20]:

# 2. MACD Strategy
def apply_macd_strategy(df):
    macd = MACD(df['close'])
    df['macd_line'] = macd.macd()
    df['macd_ema'] = df['macd_line'].ewm(span=9).mean()

    df['signal'] = 0
    df.loc[
        (df['macd_line'] > df['macd_ema']) &
        (df['macd_line'].shift() <= df['macd_ema'].shift()), 'signal'
    ] = 1
    df.loc[
        (df['macd_line'] < df['macd_ema']) &
        (df['macd_line'].shift() >= df['macd_ema'].shift()), 'signal'
    ] = -1
    return df


In [21]:

# 3. RSI + EMA Strategy
def apply_rsi_ema_strategy(df):
    df['rsi'] = RSIIndicator(df['close'], window=14).rsi()
    df['ema_21'] = df['close'].ewm(span=21).mean()

    df['signal'] = 0
    df.loc[
        (df['rsi'] > 30) & (df['rsi'].shift() <= 30) &
        (df['close'] > df['ema_21']), 'signal'
    ] = 1
    df.loc[
        ((df['rsi'] < 70) & (df['rsi'].shift() >= 70)) |
        (df['close'] < df['ema_21']), 'signal'
    ] = -1
    return df


In [22]:

# 4. Backtesting Function
def backtest_strategy(df, strategy_name):
    trades = []
    in_position = False
    entry_price = 0
    entry_time = None

    for i in range(len(df)):
        signal = df['signal'].iloc[i]
        time = df.index[i]
        price = df['close'].iloc[i]

        if signal == 1 and not in_position:
            in_position = True
            entry_price = price
            entry_time = time

        elif signal == -1 and in_position:
            exit_price = price
            exit_time = time
            pnl = exit_price - entry_price
            status = "Win" if pnl > 0 else "Loss"
            trades.append([
                entry_time, entry_price,
                exit_time, exit_price,
                strategy_name, pnl, status
            ])
            in_position = False

    return pd.DataFrame(trades, columns=[
        "Entry Time", "Entry Price", "Exit Time",
        "Exit Price", "Strategy", "PnL", "Status"
    ])

# 5. Pipeline
if __name__ == "__main__":
    df = fetch_binance_data()

    # Strategy 1
    macd_df = apply_macd_strategy(df.copy())
    macd_trades = backtest_strategy(macd_df, "MACD-EMA")

    # Strategy 2
    rsi_df = apply_rsi_ema_strategy(df.copy())
    rsi_trades = backtest_strategy(rsi_df, "RSI-EMA")

    # Combine & Export
    all_trades = pd.concat([macd_trades, rsi_trades])
    print(all_trades)
    all_trades.to_csv("strategy_backtest_results.csv", index=False)


            Entry Time  Entry Price           Exit Time  Exit Price  Strategy  \
0  2025-06-02 21:37:00    104745.93 2025-06-02 22:11:00   105129.88  MACD-EMA   
1  2025-06-02 22:14:00    105228.43 2025-06-02 22:33:00   105677.10  MACD-EMA   
2  2025-06-02 22:52:00    105684.74 2025-06-02 22:56:00   105659.68  MACD-EMA   
3  2025-06-02 23:02:00    105772.41 2025-06-02 23:05:00   105666.48  MACD-EMA   
4  2025-06-02 23:20:00    105847.39 2025-06-02 23:27:00   105786.42  MACD-EMA   
5  2025-06-02 23:44:00    105905.66 2025-06-02 23:53:00   105879.35  MACD-EMA   
6  2025-06-03 00:09:00    105875.05 2025-06-03 00:27:00   106122.02  MACD-EMA   
7  2025-06-03 00:46:00    106316.65 2025-06-03 00:52:00   106248.17  MACD-EMA   
8  2025-06-03 01:02:00    106311.44 2025-06-03 01:11:00   106202.68  MACD-EMA   
9  2025-06-03 01:21:00    106275.93 2025-06-03 01:32:00   106242.30  MACD-EMA   
10 2025-06-03 01:47:00    106289.37 2025-06-03 02:06:00   106313.91  MACD-EMA   
11 2025-06-03 02:31:00    10